<a href="https://colab.research.google.com/github/PMartey/KaggleX-Project/blob/main/Evaluating_model_STI_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.1/644.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 106.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.

import keras
import keras_nlp

import numpy as np
import pandas as pd

In [ ]:
# Load the fine tuned model that was just uploaded to Kaggle

kaggle_username = "dedemartey"
finetuned_model = keras_nlp.models.CausalLM.from_preset(f"kaggle://{kaggle_username}/gemma/keras/stibot_my_gemma2_pt")



100%|██████████| 785/785 [00:00<00:00, 687kB/s]


100%|██████████| 2.91k/2.91k [00:00<00:00, 3.37MB/s]


100%|██████████| 4.04M/4.04M [00:00<00:00, 9.28MB/s]


100%|██████████| 9.34G/9.34G [03:47<00:00, 44.1MB/s]


#Inference after fine-tuning  
After fine-tuning, responses follow the instruction provided in the prompt.


#Loss Harvesting Prompt

In [ ]:
test_df= pd.read_csv('/content/test1_data.csv')
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  43 non-null     object
 1   answer    43 non-null     object
dtypes: object(2)
memory usage: 816.0+ bytes


In [ ]:
test_samp =test_df.sample(10)

3. Evaluate using Quantitative Metrics  
BLEU Score: Measures the similarity between generated answers and reference answers by comparing n-grams. BLEU is widely used for translation and question-answering tasks, but note that it’s biased toward exact matches.  
ROUGE Score: Compares overlapping n-grams, especially for longer text. ROUGE is particularly useful for assessing responses with more paraphrasing.  
Exact Match (EM): Measures the percentage of predictions that exactly match the ground truth.  
F1 Score: Useful when exact wording isn’t required. It’s a harmonic mean of precision and recall, especially effective for partial matches.   


In [ ]:
# Example: Generate predictions for each question in the test set
predictions = []
for question in test_samp['question']:
    generated_answer = finetuned_model.generate(question)
    predictions.append(generated_answer)


In [ ]:
!pip install nltk

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=ea4e10cb6c20e0f89b696ac62aae00a19f9d5414fe89fb1f08d00797ae2df13a
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer

# Calculate BLEU and ROUGE for each generated answer
bleu_scores = []
# Remove the smoothing_function argument from RougeScorer initialization
rouge = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge_scores = []

for ref, pred in zip(test_samp['answer'], predictions):
    # Use smoothing function for BLEU calculation
    bleu_scores.append(sentence_bleu([ref.split()], pred.split(), smoothing_function=SmoothingFunction().method1))
    rouge_scores.append(rouge.score(ref, pred))

# Reason for using the smoothing function

If your model produces outputs that differ significantly from the reference answers, even if they capture the general idea, the BLEU score might still be low due to this lack of n-gram overlap. Here’s how you can handle it:

Lower the n-gram order: Reduce the BLEU score calculation to 1-gram or 2-gram BLEU, which focuses more on individual word overlaps rather than multi-word phrases.

Use a smoothing function: Applying smoothing (e.g., SmoothingFunction().method1 in NLTK) can help avoid zero scores, especially for shorter sentences or those with uncommon n-grams.

In [ ]:
# Calculate average ROUGE scores
rouge1_scores = [score['rouge1'].fmeasure for score in rouge_scores]
rougeL_scores = [score['rougeL'].fmeasure for score in rouge_scores]

avg_rouge1 = sum(rouge1_scores) / len(rouge1_scores)
avg_rougeL = sum(rougeL_scores) / len(rougeL_scores)

print(f"Average ROUGE-1: {avg_rouge1}")

Average ROUGE-1: 0.14109204268148368


In [ ]:
# Calculate average BLEU Scores
avg_bleu = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU Score: {avg_bleu}")

Average BLEU Score: 0.014291633335491421


In [ ]:
rouge_scores

[{'rouge1': Score(precision=0.07326007326007326, recall=0.7407407407407407, fmeasure=0.13333333333333336),
  'rougeL': Score(precision=0.06227106227106227, recall=0.6296296296296297, fmeasure=0.11333333333333333)},
 {'rouge1': Score(precision=0.04600484261501211, recall=0.3584905660377358, fmeasure=0.0815450643776824),
  'rougeL': Score(precision=0.0387409200968523, recall=0.3018867924528302, fmeasure=0.06866952789699571)},
 {'rouge1': Score(precision=0.03389830508474576, recall=0.7058823529411765, fmeasure=0.0646900269541779),
  'rougeL': Score(precision=0.031073446327683617, recall=0.6470588235294118, fmeasure=0.05929919137466308)},
 {'rouge1': Score(precision=0.044270833333333336, recall=0.4857142857142857, fmeasure=0.081145584725537),
  'rougeL': Score(precision=0.044270833333333336, recall=0.4857142857142857, fmeasure=0.081145584725537)},
 {'rouge1': Score(precision=0.03140096618357488, recall=0.26, fmeasure=0.05603448275862069),
  'rougeL': Score(precision=0.028985507246376812, r

 Let's break down each part of the rouge scores for my 1st question sample:

#### ROUGE-1: Measures the overlap of unigrams


(individual words) between the model-generated summary and the reference summary.

Precision (0.0733): Around 7.3% of the words in the generated summary match words in the reference summary.  
Recall (0.7407): Around 74.1% of the words in the reference summary are found in the generated summary, which is relatively high.  
F-measure (0.1333): The harmonic mean of precision and recall for ROUGE-1.   
The low value indicates a significant disparity between recall and precision, suggesting the generated summary includes a lot of additional content not in the reference.

###ROUGE-L: Measures the longest matching sequence of words (or n-grams), which captures the fluency or structure similarity between the summaries.

Precision (0.0623): About 6.2% of the sequences in the generated summary align with those in the reference.  
Recall (0.6296): Around 63% of the sequences in the reference appear in the generated summary, again showing a high recall.   
F-measure (0.1133): Like with ROUGE-1, the F-measure here is low, reflecting that although there are matching sequences, the generated summary includes non-matching phrases.  

Interpretation: The high recall and low precision indicate that your generated summary is including most of the reference content but with additional words or phrases, potentially making it longer and less precise. Adjustments to the model to improve precision could help make the generated summary more concise and closely aligned with the reference.

To improve the finetuned GEMMA model's precision for text summarization, here are some strategies we will try in future:

1. Adjust Training Data: If your dataset includes long summaries, consider creating shorter reference summaries with key points only. This adjustment encourages the model to generate more concise outputs, helping reduce irrelevant content.

2. Fine-Tuning with Penalization for Redundancy: Modify the loss function to penalize redundant or non-matching n-grams. This technique helps the model learn to avoid unnecessary words that don’t align with the reference summary.

3. Adjust Decoding Parameters: Experiment with decoding strategies like:

- Beam Search with Length Penalty: Add a length penalty during beam search to discourage overly long summaries. This parameter helps to balance recall and precision by limiting the number of tokens generated.
- Top-k Sampling and Top-p (Nucleus) Sampling: These sampling techniques can make the summary more targeted, helping reduce repetitive or filler content.
4. Add Precision-Optimized Loss Terms: Introduce a precision-oriented loss term that rewards token matches while discouraging extra, non-matching tokens in generated summaries.

5. Data Augmentation with Negative Samples: Train the model on "negative samples" or incorrect summaries that contain excessive or irrelevant content. This can help the model learn to distinguish between essential and non-essential phrases.

6. ROUGE-Precision-Centered Post-processing: After generating a summary, apply a post-processing step to remove any non-essential tokens or phrases that do not contribute to higher ROUGE precision.

Testing and comparing outputs after each adjustment will help pinpoint which techniques best align with your goal of increasing precision without compromising summary quality.